In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Sequential model is appropritate for a stack of layers where each layer
# has exactly one input and one output tensor

model = keras.Sequential([
    layers.Dense(2, activation='relu', name='layer1'),
    layers.Dense(3, activation='relu', name='layer2'),
    layers.Dense(4, name='layer3')
])

x = tf.ones((3, 3))
y = model(x)

# Creating sequential model

In [3]:
# use the Sequential() constructor
model = keras.Sequential([
    layers.Dense(2, activation='relu'),
    layers.Dense(3, activation='relu'),
    layers.Dense(4)
])

In [4]:
# can access layers throught attribute
model.layers

In [5]:
# can also use model.add(), this is equivalent
model = keras.Sequential()
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(4))

In [6]:
# can use pop to remove layer
model.pop()
len(model.layers)

2

In [7]:
# use name argument to annotate layers
model = keras.Sequential(name='my_sequential')
model.add(layers.Dense(2, activation='relu', name='layer1'))
model.add(layers.Dense(3, activation='relu', name='layer2'))
model.add(layers.Dense(4, name='layer3'))

# Specify input shape

In [8]:
# keras layer without input shape doesn't have weights
layer = layers.Dense(3)
layer.weights

[]

In [9]:
# it will create weights when it is called with an input
x = tf.ones((1, 4))
y = layer(x)
layer.weights

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.7784909 ,  0.922567  ,  0.0332675 ],
        [-0.04395431,  0.36952293,  0.31643713],
        [-0.27970302, -0.04569477,  0.8854706 ],
        [ 0.6030197 , -0.8739552 , -0.07982934]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [10]:
# same with sequential models
model = keras.Sequential([
    layers.Dense(2, activation='relu'),
    layers.Dense(3, activation='relu'),
    layers.Dense(4)
]) # no weights at this stage because no input is specified
# no model.weights nor model.summary()

# call model with input, in order to build it
x = tf.ones((1, 4))
y = model(x)
len(model.weights)

6

In [11]:
# can also call summary now
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (1, 2)                    10        
_________________________________________________________________
dense_8 (Dense)              (1, 3)                    9         
_________________________________________________________________
dense_9 (Dense)              (1, 4)                    16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [12]:
# in order to see model.summary() without inputting data, use a Input()
model = keras.Sequential()
model.add(keras.Input(shape=(4, )))
model.add(layers.Dense(2, activation='relu'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Input() is not a layer
model.layers

In [14]:
# or use argument to first layer
model = keras.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4, )))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


# Add() + summary()

In [15]:
# can use add() + summary() to visualize model building along the way
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))
model.add(layers.Conv2D(32, 5, strides=2, activation='relu'))
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D(3))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________


In [16]:
# output is (40, 40, 32), can keep downsampling
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 32)       

In [17]:
# output is (4, 4, 32), can use global max pooling
model.add(layers.GlobalMaxPooling2D())

model.add(layers.Dense(10))

# Feature extraction

In [18]:
initial_model = keras.Sequential([
    keras.Input(shape=(250, 250, 3)),
    layers.Conv2D(32, 5, strides=2, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu')
])

# can use each layer's input and output attributes to extract feature
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers]
)

x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [19]:
# or extract feature from one layer
initial_model = keras.Sequential([
    keras.Input(shape=(250, 250, 3)),
    layers.Conv2D(32, 5, strides=2, activation='relu'),
    layers.Conv2D(32, 3, activation='relu', name='my_intermediate_layer'),
    layers.Conv2D(32, 3, activation='relu')
])

feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name='my_intermediate_layer').output
)

x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

# Transfer learning

In [21]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])

# load pretrained weights
# model.load_weights(...)

# freeze all layers except last layer
for layer in model.layers[:-1]:
    layer.trainable = False

# recompile and train
# model.compile(...)
# model.train(...)

In [22]:
# alternatively, add new classification layers to a existing model
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg'
)

# freeze base model
base_model.trainable = False

# use Sequential model to add trainable layers
model = keras.Sequential([
    base_model,
    layers.Dense(1000)
])

# recompile and train
# model.compile(...)
# model.train(...)

13582336/83683744 [===>..........................] - ETA: 24s

KeyboardInterrupt: 